In [63]:
import pandas as pd
from datetime import datetime, timedelta, date
import io

ST_2015=pd.read_excel('data/states/Nevada/Nevadadeaths.xlsx',sheet_name='2015F')
ST_2015['Year'] = 2015
ST_2016=pd.read_excel('data/states/Nevada/Nevadadeaths.xlsx',sheet_name='2016F')
ST_2016['Year'] = 2016
ST_2017=pd.read_excel('data/states/Nevada/Nevadadeaths.xlsx',sheet_name='2017F')
ST_2017['Year'] = 2017
ST_2018=pd.read_excel('data/states/Nevada/Nevadadeaths.xlsx',sheet_name='2018F')
ST_2018['Year'] = 2018
ST_2019=pd.read_excel('data/states/Nevada/Nevadadeaths.xlsx',sheet_name='2019F')
ST_2019['Year'] = 2019
ST_2020=pd.read_excel('data/states/Nevada/Nevadadeaths.xlsx',sheet_name='2020F')
ST_2020['Year'] = 2020
us_state_df = ST_2015
us_state_df = pd.concat([us_state_df, ST_2016]).reset_index(drop=True) 
us_state_df = pd.concat([us_state_df, ST_2017]).reset_index(drop=True) 
us_state_df = pd.concat([us_state_df, ST_2018]).reset_index(drop=True)  
us_state_df = pd.concat([us_state_df, ST_2019]).reset_index(drop=True) 
us_state_df = pd.concat([us_state_df, ST_2020]).reset_index(drop=True) 

county_pop_df = pd.read_csv("data/states/county_pop.csv", encoding = "utf-8",error_bad_lines=True)
county_pop_df = county_pop_df[county_pop_df['STNAME'] == 'Nevada']

county_fips_df = pd.read_csv("data/states/county_fips.csv", error_bad_lines=True)
county_fips_df = county_fips_df[county_fips_df['STNAME'] == 'Nevada']
#county_fips_df["CTYNAME"] = county_fips_df["CTYNAME"].str.upper()
county_fips_dict = dict(zip(county_fips_df.CTYNAME,county_pop_df.COUNTY))
us_state_df

,Week,Place of Death County,Deaths,Year
0,1,Carson City,16,2015
1,2,Carson City,14,2015
2,3,Carson City,15,2015
3,4,Carson City,17,2015
4,5,Carson City,17,2015
...,...,...,...,...
4823,19,White Pine,1,2020
4824,20,White Pine,1,2020
4825,21,White Pine,3,2020
4826,22,White Pine,0,2020


In [64]:

us_state_constant_df = us_state_df.copy()
us_state_constant_df['state_FIPS'] = 32
us_state_constant_df['County'] = 0
us_state_constant_df['population_adjustment'] = 'Constant'
us_state_constant_df['pop'] = 0
us_state_constant_df["total_deaths_per100k"] = 0
us_state_constant_df["excess_deaths_per100k"] = 0.0
county_2017_pop= dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2017))
pop_100k = 100000.0
for index, row in us_state_constant_df.iterrows():
    county_name = row['Place of Death County']
    pop = county_2017_pop[county_fips_dict[county_name]]
    us_state_constant_df.at[index, 'County'] = county_fips_dict[county_name]
    us_state_constant_df.at[index,'pop'] = pop
    us_state_constant_df.at[index, 'total_deaths_per100k'] = (row["Deaths"] *pop_100k )/(pop* 1.0)



avg_deaths = us_state_constant_df[us_state_constant_df['Year'] < 2020].groupby(['County', 'Week'])['total_deaths_per100k'].mean()
avg_deaths_df = avg_deaths.to_frame(name = 'avg_deaths').reset_index()

for index, row in us_state_constant_df.iterrows():
    year = row['Year']
    county = row['County']
    week = row['Week']
    if year == 2020:
        avgd = avg_deaths_df[(avg_deaths_df['County'] == county) & (avg_deaths_df['Week'] == week)]
        if not avgd.empty:
            us_state_constant_df.at[index, "excess_deaths_per100k"] = round(row['total_deaths_per100k'] - avgd['avg_deaths'], 4)


In [65]:
county_year_dict = dict()
county_year_dict[2014] = dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2014))
county_year_dict[2015] = dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2015))
county_year_dict[2016] = dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2016))
county_year_dict[2017] = dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2017))
county_year_dict[2018] = dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2018))
county_year_dict[2019] = dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2019))

us_state_lagged_df = us_state_df.copy()
us_state_lagged_df['state_FIPS'] = 32
us_state_lagged_df['population_adjustment'] = 'Lagged'
us_state_lagged_df['pop'] = 0
us_state_lagged_df["total_deaths_per100k"] = 0
us_state_lagged_df["excess_deaths_per100k"] = 0.0
pop_100k = 100000.0
for index, row in us_state_lagged_df.iterrows():
    year = row['Year'] - 1  
    county_name = row['Place of Death County']
    county_pop_dict = county_year_dict[year]
    pop = county_pop_dict[county_fips_dict[county_name]]
    us_state_lagged_df.at[index, 'County'] = county_fips_dict[county_name]
    
    us_state_lagged_df.at[index,'pop'] = pop
    us_state_lagged_df.at[index, 'total_deaths_per100k'] = (row["Deaths"] *pop_100k )/(pop* 1.0)



avg_deaths = us_state_lagged_df[us_state_lagged_df['Year'] < 2020].groupby(['County', 'Week'])['total_deaths_per100k'].mean()
avg_deaths_df = avg_deaths.to_frame(name = 'avg_deaths').reset_index()

for index, row in us_state_lagged_df.iterrows():
    year = row['Year']
    county = row['County']
    week = row['Week']
    if year == 2020:
        avgd = avg_deaths_df[(avg_deaths_df['County'] == county) & (avg_deaths_df['Week'] == week)]
        if not avgd.empty:
            us_state_lagged_df.at[index, "excess_deaths_per100k"] = round(row['total_deaths_per100k'] - avgd['avg_deaths'],4)       

us_state_merged_df = pd.concat([us_state_constant_df, us_state_lagged_df])
us_state_merged_df = us_state_merged_df.sort_values(['County', 'Week', 'population_adjustment','Year'])


In [66]:
us_state_panel_df = pd.DataFrame(columns=['State_FIPS','County_FIPS','Year', 'Week','Population_Adjustment', 
                                        'Deaths_Per100k', 'Excess_Deaths_Per100k'])        
us_state_panel_df['State_FIPS']  = us_state_merged_df['state_FIPS']
us_state_panel_df['County_FIPS'] = us_state_merged_df['County']
us_state_panel_df['Year']  = us_state_merged_df['Year']
us_state_panel_df['Week'] = us_state_merged_df['Week']
us_state_panel_df['Population_Adjustment'] = us_state_merged_df['population_adjustment']
us_state_panel_df['Deaths_Per100k'] = us_state_merged_df['total_deaths_per100k']
us_state_panel_df['Excess_Deaths_Per100k'] = round(us_state_merged_df['excess_deaths_per100k'],4)

In [67]:
us_state_panel_df.to_csv("nevada_panel.csv",index=False)

In [62]:
us_state_panel_df[us_state_panel_df['Year'] == 2020]

,State_FIPS,County_FIPS,Year,Week,Population_Adjustment,Deaths_Per100k,Excess_Deaths_Per100k
4460,32,1.0,2020,1,Constant,16,0.0
4460,32,1.0,2020,1,Lagged,16,-0.2
4461,32,1.0,2020,2,Constant,29,6.8
4461,32,1.0,2020,2,Lagged,28,5.8
4462,32,1.0,2020,3,Constant,20,-0.8
...,...,...,...,...,...,...,...
4457,32,510.0,2020,21,Lagged,32,5.0
4458,32,510.0,2020,22,Constant,33,10.2
4458,32,510.0,2020,22,Lagged,32,9.2
4459,32,510.0,2020,23,Constant,27,3.0
